<a href="https://colab.research.google.com/github/maxruther/HCP_Fraud_Detection/blob/main/analysis_in_segments/HPFD_3_FeatureEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Feature Engineering**
## Healthcare Provider Fraud Detection - **Part 3**

In this third segment of my claim fraud analysis, I employ aggregative feature engineering to create provider-level attributes from the claim-level data. This is a first step in creating a dataset of items that only represents providers, for classification thereof.

This document's main areas concern:
- [Claim-related aggregations by provider](#claim-related-aggs)
- [Patient-related aggregations by provider](#patient-related-aggs)
  - [Prevalences of chronic conditions](#chron-cond-prevs)
  - [Average amounts of patient reimbursements and deductibles](#avg-ins-amts)


<br></br>

### **Overview**

With the various predictor datasets now combined, the focus of my data integration shifts from merging to aggregation.

By aggregating many of the existing attributes by provider, I engineer features that are at the same level as the labels. These aggregate features will exclusively form the training data for my classification models.

Each provider-level aggregation relates to either **patient** or **claim** information. The **claim** aggregations involve their counts, amounts, and lengths.

The **patient** aggregations concern one of the following:
1. The **percentage prevalence of chronic conditions** among a provider's patients.
2. A provider's patients' **average annual reimbursement and deductible amounts**, meaning those of their insurance coverage.

Every attribute created in this section will be named with the prefix "Prv_". This is to indicate that it is a provider-level aggregation.

### **Quick Setup**



**Importing libraries**

In [ ]:
import pandas as pd
import numpy as np

import warnings

**Loading objects from the preceding part**

In [ ]:
# Mounting my Google Drive, where I've saved the preceding part's objects to file:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Loading the object necessary for this part

# Project directory path in Google Drive
project_dir_path = '/content/gdrive/MyDrive/fraud_data_dsc540/'

# Filepath to files saved in part 2.
part2_filepath = project_dir_path + '/walkthrough/part_2/'

df = pd.read_pickle(f'{part2_filepath}df.pkl')

<a name="claim-related-aggs"></a>
### **Claim-related aggregations by provider**



#### **Total Claim Count**
#### *Prv_Claim_Count*

Calculating the total counts of claims, by provider:

In [ ]:
df.groupby('Provider').size().head(3)

,0
Provider,
PRV51001,25
PRV51003,132
PRV51004,149


In [ ]:
df['Prv_Claim_Count'] = df.groupby('Provider')['ClaimID'].transform('count')

##### *peek_df_for_new_attr()*

Often when I create an attribute, I like to eyeball the success of its creation by taking quick "peek" at the transformed dataset. To that end, below I define a function *peek_df_for_new_attr*.

Given the name of the new attribute, it returns its  values for the first few listed providers, those listed in the code cell two previous ('PRV51001', 'PRV51003', and 'PRV51004'.)

In [ ]:
def peek_df_for_new_attr(new_attr):
  # Create a list of the first three
  first_few_prvs = list(df.groupby('Provider').size().head().index[:3].values)
  clm_and_prvIDs = ['ClaimID', 'Provider']

  # Following is a chain of query clauses, effectively:

  # Suppress a warning about reindexing dangers (I'm only querying to view, so
  # I feel somewhat comfortable with this choice.)
  with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    # 1. Drop duplicates from df (temporarily)
    result_df = df.drop_duplicates(subset='Provider')\
    [df['Provider'].isin(first_few_prvs)]\
    [clm_and_prvIDs + [new_attr]]\
    .sort_values(by='Provider')

  return result_df


In [ ]:
peek_df_for_new_attr('Prv_Claim_Count')

,ClaimID,Provider,Prv_Claim_Count
2572,CLM733300,PRV51001,25
1056,CLM374220,PRV51003,132
4172,CLM264498,PRV51004,149


#### **Total Claim Amount**
#### *Prv_Claim_AmtTotal*

Calculating the total amount reimbursed for claims, by provider.

In [ ]:
df.groupby('Provider').sum('InscClaimAmtReimbursed')['InscClaimAmtReimbursed'].head(3)

,InscClaimAmtReimbursed
Provider,
PRV51001,104640
PRV51003,605670
PRV51004,52170


In [ ]:
df['Prv_Claim_AmtTotal'] = df.groupby('Provider')['InscClaimAmtReimbursed'].transform('sum')

Peeking at `df` to see the implemented change:

In [ ]:
peek_df_for_new_attr('Prv_Claim_AmtTotal')

,ClaimID,Provider,Prv_Claim_AmtTotal
2572,CLM733300,PRV51001,104640
1056,CLM374220,PRV51003,605670
4172,CLM264498,PRV51004,52170


#### **Average Claim Amount**
#### *Prv_Claim_AmtAvg*

Calculating the average amount reimbursed for claims, by provider.



In [ ]:
df.groupby('Provider').mean('InscClaimAmtReimbursed')['InscClaimAmtReimbursed'].head(3)

,InscClaimAmtReimbursed
Provider,
PRV51001,4185.600000
PRV51003,4588.409091
PRV51004,350.134228


In [ ]:
df['Prv_Claim_AmtAvg'] = df.groupby('Provider')['InscClaimAmtReimbursed'].transform('mean')

Peeking at `df` to see the implemented change:

In [ ]:
peek_df_for_new_attr('Prv_Claim_AmtAvg')

,ClaimID,Provider,Prv_Claim_AmtAvg
2572,CLM733300,PRV51001,4185.600000
1056,CLM374220,PRV51003,4588.409091
4172,CLM264498,PRV51004,350.134228


#### **Inpatient Share of Claims**
#### *Prv_Claim_IPShare*

Calculating the share of claims that are associated with inpatients, by provider.

In [ ]:
df['PatType'].value_counts()

,count
PatType,
Outpatient,517737
Inpatient,40474


In [ ]:
df['isInpatient'] = pd.factorize(df['PatType'])[0]
df[['PatType','isInpatient']].value_counts()

,,count
PatType,isInpatient,
Outpatient,0,517737
Inpatient,1,40474


In [ ]:
df.groupby('Provider').mean('isInpatient')['isInpatient'].head(3)

,isInpatient
Provider,
PRV51001,0.200000
PRV51003,0.469697
PRV51004,0.000000


In [ ]:
df['Prv_Claim_IPShare'] = df.groupby('Provider')['isInpatient'].transform('mean')

Peeking at `df` to see the implemented change:

In [ ]:
peek_df_for_new_attr('Prv_Claim_IPShare')

,ClaimID,Provider,Prv_Claim_IPShare
2572,CLM733300,PRV51001,0.200000
1056,CLM374220,PRV51003,0.469697
4172,CLM264498,PRV51004,0.000000


#### **Average Claim Length**

#### *Prv_Claim_LengthAvg*

Calculating the average length of claims, by provider.

Claim length isn't a given attribute in this data, but their start and end dates are. given data does not include a dedicated attribute for claim length, but instead gives claims' start and end dates. So I construct claim length from the difference of those dates, then average that to create an attribute for average claim length.

##### **Creating the _ClaimLength_ field**

I am creating this _ClaimLength_ attribute in the exact same way that I created the _LengthOfStay_ attribute in a prior section. (That was in the last subsection of the "Examining table relationships" section.)

I am measuring the difference in days between a claim's end date and its start date.

In [ ]:
df[['ClaimStartDt', 'ClaimEndDt']][df['ClaimStartDt'] < df['ClaimEndDt']].head(3)

,ClaimStartDt,ClaimEndDt
8,2009-04-25,2009-05-15
9,2009-05-04,2009-05-16
21,2009-09-30,2009-10-20


In [ ]:
df['ClaimLength'] = (pd.to_datetime(df['ClaimEndDt']) - pd.to_datetime(df['ClaimStartDt'])).dt.days

In [ ]:
df[['ClaimStartDt', 'ClaimEndDt', 'ClaimLength']][df['ClaimStartDt'] < df['ClaimEndDt']].head(3)

,ClaimStartDt,ClaimEndDt,ClaimLength
8,2009-04-25,2009-05-15,20
9,2009-05-04,2009-05-16,12
21,2009-09-30,2009-10-20,20


Checking for any negative _ClaimLength_ values, which would strike me as invalid:

In [ ]:
df[df['ClaimLength'] < 0]

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Prv_Claim_Count,Prv_Claim_AmtTotal,Prv_Claim_AmtAvg,isInpatient,Prv_Claim_IPShare,ClaimLength


No negative *ClaimLength* values.

In [ ]:
df[df['ClaimLength'] >= 0].shape[0] == df.shape[0]

True

##### **Averaging *ClaimLength* by Provider**

In [ ]:
df.groupby('Provider').mean('ClaimLength')['ClaimLength'].head(3)

,ClaimLength
Provider,
PRV51001,1.440000
PRV51003,3.674242
PRV51004,1.429530


In [ ]:
df['Prv_Claim_LengthAvg'] = df.groupby('Provider')['ClaimLength'].transform('mean')

Peeking at `df` to see the implemented change:

In [ ]:
peek_df_for_new_attr('Prv_Claim_LengthAvg')

,ClaimID,Provider,Prv_Claim_LengthAvg
2572,CLM733300,PRV51001,1.440000
1056,CLM374220,PRV51003,3.674242
4172,CLM264498,PRV51004,1.429530


This was the last of the claim-related aggregations. Next, I turn to those relating to patients.

<a name="patient-related-aggs"></a>
### **Patient-related aggregations by provider**

My aggregation process in this section differs slightly than in that preceding. Instead of creating one aggregate feature at a time, as I did for the claim-related ones, I here create multiple aggregate features at once.

To execute this for both subject areas of my patient-related aggregation, _Chronic Condition_ and _Insurance Amount_, I take the following steps:
1. Identify the relevant attributes in a list, by a common pattern in their names.
2. For the aggregation, group by both provider *and patient*, then by provider again, then take the averages.
3. Rename these transformed, aggregate attributes to reflect the change.
4. Join these to the main dataset, by the _Provider_ attribute.






<a name="chron-cond-prevs"></a>
#### **Chronic Condition Prevalence**

This data includes many flags indicating whether a patient suffers from a given chronic condition. With this first set of patient-related aggregations, I measure the percentage prevalences of those chronic conditions among each provider's patients.

This is information that once struck me as helpful in identifying or distinguishing providers, perhaps by specialty or just by focus, more informally. However, after reading in relevant literature that provider specialty was found to be an insignificant predictor of fraud, I became more pessimistic about its value.

##### **Identifying the relevant attributes**

First, identifying the chronic condition flag attributes by their prefix, 'ChronicCond_':

In [ ]:
cond_attrs = [x for x in df.columns.values.tolist() if 'ChronicCond_' in x]
cond_attrs

['ChronicCond_Alzheimer',
 'ChronicCond_Heartfailure',
 'ChronicCond_KidneyDisease',
 'ChronicCond_Cancer',
 'ChronicCond_ObstrPulmonary',
 'ChronicCond_Depression',
 'ChronicCond_Diabetes',
 'ChronicCond_IschemicHeart',
 'ChronicCond_Osteoporasis',
 'ChronicCond_rheumatoidarthritis',
 'ChronicCond_stroke']

##### **Executing the aggregation**

###### **Adjusting the _ChronicCond_ mapping**

The chronic condition attributes use the value '2' to represent that the patient does not suffer from the condition. The value '1' indicates the positive case.

I am remapping these '2' values to '0' instead. That way, if every patient is uniquely represented, I can average these values to calculate prevalence percentages.

Viewing the first few rows projected over these attributes, both before and after I implement the remapping:

In [ ]:
df[cond_attrs].head(3)

,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke
0,2,2,2,2,2,2,2,2,2,2,2
1,1,2,2,2,2,2,2,1,2,2,2
2,1,2,2,2,2,2,2,1,2,2,2


In [ ]:
df[cond_attrs] = df[cond_attrs].replace(2, 0)
df[cond_attrs].head(3)

,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke
0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,1,0,0,0


###### **Aggregating**

What I'm trying to measure is the prevalence of the given chronic conditions among each provider's patients.

To calculate this, I first group by both provider and patient, than take the maximum of those _ChronicCond_ values as a means of eliminating duplicates. Because every claim record shows the same _ChronicCond_ value for each patient, I could also take the minimum of these and get the same result.

Next, I group by provider again and take the average. By taking the average of these unique patients' binary flags, which show '1' in the positive case and '0' in the negative, I arrive at a prevalence percentage for that condition.

In [ ]:
PatCondPrevs_byPrv = df[['Provider', 'BeneID',] + cond_attrs].groupby(['Provider', 'BeneID']).max().groupby('Provider').mean()
PatCondPrevs_byPrv.head(3)

,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke
Provider,,,,,,,,,,,
PRV51001,0.583333,0.750000,0.708333,0.208333,0.375000,0.375000,0.833333,0.916667,0.250000,0.333333,0.208333
PRV51003,0.376068,0.598291,0.444444,0.085470,0.282051,0.401709,0.743590,0.846154,0.239316,0.273504,0.076923
PRV51004,0.434783,0.594203,0.340580,0.115942,0.268116,0.434783,0.695652,0.710145,0.311594,0.297101,0.115942


##### **Renaming these new aggregate features**

For my own reference in this renaming, I re-print the existing names of the chronic condition attributes:

In [ ]:
cond_attrs

['ChronicCond_Alzheimer',
 'ChronicCond_Heartfailure',
 'ChronicCond_KidneyDisease',
 'ChronicCond_Cancer',
 'ChronicCond_ObstrPulmonary',
 'ChronicCond_Depression',
 'ChronicCond_Diabetes',
 'ChronicCond_IschemicHeart',
 'ChronicCond_Osteoporasis',
 'ChronicCond_rheumatoidarthritis',
 'ChronicCond_stroke']

I next draft the new names for these attributes, ones that better reflect their new measures.

After I manually specify abbreviated descriptions of each condition, I programmatically prefix the attribute names with 'Prv_condPrev_' to indicate that they are provider-level aggregations of chronic condition prevalence:

In [ ]:
abbrvd_conds = ['Alz', 'HeartF', 'KidneyD', 'Cancer', 'ObstrP', 'Depr',
                'Diab', 'IschemicH', 'Osteo', 'Rheuma', 'Stroke']

renamedCondCols = ['Prv_condPrev_' + cond for cond in abbrvd_conds]
renamedCondCols

['Prv_condPrev_Alz',
 'Prv_condPrev_HeartF',
 'Prv_condPrev_KidneyD',
 'Prv_condPrev_Cancer',
 'Prv_condPrev_ObstrP',
 'Prv_condPrev_Depr',
 'Prv_condPrev_Diab',
 'Prv_condPrev_IschemicH',
 'Prv_condPrev_Osteo',
 'Prv_condPrev_Rheuma',
 'Prv_condPrev_Stroke']

The dataframe method 'rename' will rename variables if I pass it a dictionary mapping of the desired renamings.

Using the lists of variable names just formed, I create this mapping by entering them as arguments in a combined call to the zip() and dict() methods:

In [ ]:
condRenaming_dict = dict(zip(cond_attrs, renamedCondCols))
condRenaming_dict

{'ChronicCond_Alzheimer': 'Prv_condPrev_Alz',
 'ChronicCond_Heartfailure': 'Prv_condPrev_HeartF',
 'ChronicCond_KidneyDisease': 'Prv_condPrev_KidneyD',
 'ChronicCond_Cancer': 'Prv_condPrev_Cancer',
 'ChronicCond_ObstrPulmonary': 'Prv_condPrev_ObstrP',
 'ChronicCond_Depression': 'Prv_condPrev_Depr',
 'ChronicCond_Diabetes': 'Prv_condPrev_Diab',
 'ChronicCond_IschemicHeart': 'Prv_condPrev_IschemicH',
 'ChronicCond_Osteoporasis': 'Prv_condPrev_Osteo',
 'ChronicCond_rheumatoidarthritis': 'Prv_condPrev_Rheuma',
 'ChronicCond_stroke': 'Prv_condPrev_Stroke'}

I use the `rename()` method to apply this mapping to my temporary dataframe of prevalence percentages, `PatCondPrevs_byPrv`:

In [ ]:
PatCondPrevs_byPrv.rename(columns=condRenaming_dict, inplace=True)
PatCondPrevs_byPrv.head(3)

,Prv_condPrev_Alz,Prv_condPrev_HeartF,Prv_condPrev_KidneyD,Prv_condPrev_Cancer,Prv_condPrev_ObstrP,Prv_condPrev_Depr,Prv_condPrev_Diab,Prv_condPrev_IschemicH,Prv_condPrev_Osteo,Prv_condPrev_Rheuma,Prv_condPrev_Stroke
Provider,,,,,,,,,,,
PRV51001,0.583333,0.750000,0.708333,0.208333,0.375000,0.375000,0.833333,0.916667,0.250000,0.333333,0.208333
PRV51003,0.376068,0.598291,0.444444,0.085470,0.282051,0.401709,0.743590,0.846154,0.239316,0.273504,0.076923
PRV51004,0.434783,0.594203,0.340580,0.115942,0.268116,0.434783,0.695652,0.710145,0.311594,0.297101,0.115942


##### **Joining these features to the main dataset**

I complete this aggregation over patients' chronic conditions by integrating the attributes to the working dataset, via an inner join.

In [ ]:
df = df.merge(PatCondPrevs_byPrv, how='inner', on='Provider')

In [ ]:
df[['BeneID', 'ClaimID', 'Provider'] + renamedCondCols].head()

,BeneID,ClaimID,Provider,Prv_condPrev_Alz,Prv_condPrev_HeartF,Prv_condPrev_KidneyD,Prv_condPrev_Cancer,Prv_condPrev_ObstrP,Prv_condPrev_Depr,Prv_condPrev_Diab,Prv_condPrev_IschemicH,Prv_condPrev_Osteo,Prv_condPrev_Rheuma,Prv_condPrev_Stroke
0,BENE11002,CLM624349,PRV56011,0.359966,0.510309,0.316151,0.128007,0.256873,0.392612,0.628866,0.703608,0.281787,0.273196,0.072165
1,BENE11003,CLM189947,PRV57610,0.562500,0.437500,0.375000,0.187500,0.250000,0.187500,0.687500,0.812500,0.187500,0.375000,0.000000
2,BENE11003,CLM438021,PRV57595,0.375000,0.500000,0.333333,0.208333,0.375000,0.250000,0.666667,0.708333,0.375000,0.375000,0.041667
3,BENE11004,CLM121801,PRV56011,0.359966,0.510309,0.316151,0.128007,0.256873,0.392612,0.628866,0.703608,0.281787,0.273196,0.072165
4,BENE11004,CLM150998,PRV56011,0.359966,0.510309,0.316151,0.128007,0.256873,0.392612,0.628866,0.703608,0.281787,0.273196,0.072165


<a name="avg-ins-amts"></a>
#### **Average Amounts of the Patients' Reimbursements and Deductibles**

Though they relate to claims, patients' annual reimbursement amounts and deductibles are also featured in the beneficiary/patient file. For each patient, these measures are separately taken for their inpatient and outpatient costs.



##### **Identifying the relevant attributes**

First, identifying the pertinent attributes. A variable name containing 'Annual' as a substring indicates it as one of these coverage-related fields of interest.

Creating a list of these attributes' names:

In [ ]:
pat_ins_amt_attrs = [x for x in df.columns.tolist() if 'Annual' in x]
pat_ins_amt_attrs

['IPAnnualReimbursementAmt',
 'IPAnnualDeductibleAmt',
 'OPAnnualReimbursementAmt',
 'OPAnnualDeductibleAmt']

##### **Executing the aggregation**



Conveniently, this aggregation is done in the exact same way as that in the previous section, even though we are now aggregating over integers instead of binary flags.

The aggregation, coded below, involves these two steps:
1. Group by provider and patient and take the maximums of the insurance amount attributes, to deduplicate.
2. Group those by provider and take their average.


In [ ]:
patInsAmtByPrv_df = df[['Provider', 'BeneID'] + pat_ins_amt_attrs].groupby(['Provider', 'BeneID']).max().groupby('Provider').mean()
patInsAmtByPrv_df.head(3)

,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
Provider,,,,
PRV51001,18047.916667,890.000000,2537.500000,474.916667
PRV51003,6814.017094,822.632479,2490.598291,664.529915
PRV51004,4596.739130,454.144928,2095.144928,600.869565


##### **Renaming these latest aggregate features**

I print the existing names of the attributes for my reference in renaming:

In [ ]:
pat_ins_amt_attrs

['IPAnnualReimbursementAmt',
 'IPAnnualDeductibleAmt',
 'OPAnnualReimbursementAmt',
 'OPAnnualDeductibleAmt']

Next, I programmatically draft new names for these attributes:

In [ ]:
ins_amt_attr_renamings = []

# Programmatically draft each attr's name by parsing the existing one
for attr_name in pat_ins_amt_attrs:

  # Parse the patient type: 'IP' or 'OP'
  pat_type = attr_name[:2]

  # Parse amount type: 'Deductible' or 'Reimbursement'
  amt_type = attr_name[8:attr_name.index('Amt')]

  # Abbreviate this amount type string
  amt_type = amt_type.replace('Reimbursement', 'Reimb')
  amt_type = amt_type.replace('Deductible', 'Deduct')

  # Prepend with 'Prv_InsAmt_' and add to the list of new attr names.
  ins_amt_attr_renamings.append(f'Prv_InsAmt_{pat_type}_{amt_type}Avg')

ins_amt_attr_renamings

['Prv_InsAmt_IP_ReimbAvg',
 'Prv_InsAmt_IP_DeductAvg',
 'Prv_InsAmt_OP_ReimbAvg',
 'Prv_InsAmt_OP_DeductAvg']

Now that I have lists of both the old and new attribute names, I can create a mapping:

In [ ]:
InsAmtRenaming_dict = dict(zip(pat_ins_amt_attrs, ins_amt_attr_renamings))
InsAmtRenaming_dict

{'IPAnnualReimbursementAmt': 'Prv_InsAmt_IP_ReimbAvg',
 'IPAnnualDeductibleAmt': 'Prv_InsAmt_IP_DeductAvg',
 'OPAnnualReimbursementAmt': 'Prv_InsAmt_OP_ReimbAvg',
 'OPAnnualDeductibleAmt': 'Prv_InsAmt_OP_DeductAvg'}

I apply this mapping to the temporary dataframe holding my insurance amount aggregations:

In [ ]:
patInsAmtByPrv_df.rename(columns=InsAmtRenaming_dict, inplace=True)
patInsAmtByPrv_df.head(3)

,Prv_InsAmt_IP_ReimbAvg,Prv_InsAmt_IP_DeductAvg,Prv_InsAmt_OP_ReimbAvg,Prv_InsAmt_OP_DeductAvg
Provider,,,,
PRV51001,18047.916667,890.000000,2537.500000,474.916667
PRV51003,6814.017094,822.632479,2490.598291,664.529915
PRV51004,4596.739130,454.144928,2095.144928,600.869565


The renaming of these attributes was successful.

##### **Joining these features to the main dataset**

I complete this aggregation over patients' annual insurance amounts by integrating the attributes into the main working dataset, via an inner join.

In [ ]:
df = df.merge(patInsAmtByPrv_df, how='inner', on='Provider')
df.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,Prv_condPrev_Depr,Prv_condPrev_Diab,Prv_condPrev_IschemicH,Prv_condPrev_Osteo,Prv_condPrev_Rheuma,Prv_condPrev_Stroke,Prv_InsAmt_IP_ReimbAvg,Prv_InsAmt_IP_DeductAvg,Prv_InsAmt_OP_ReimbAvg,Prv_InsAmt_OP_DeductAvg
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,0.392612,0.628866,0.703608,0.281787,0.273196,0.072165,3873.187285,373.103093,1539.381443,447.079038
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,...,0.187500,0.687500,0.812500,0.187500,0.375000,0.000000,2149.375000,333.750000,1606.250000,640.000000
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,...,0.250000,0.666667,0.708333,0.375000,0.375000,0.041667,2429.166667,267.000000,1309.166667,410.000000
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,0.392612,0.628866,0.703608,0.281787,0.273196,0.072165,3873.187285,373.103093,1539.381443,447.079038
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,0.392612,0.628866,0.703608,0.281787,0.273196,0.072165,3873.187285,373.103093,1539.381443,447.079038


### **Discarding the *Claim Procedure Code* attributes**

The patient-level focus of the preceding aggregations led me to next consider yet another group of attributes originated in the beneficiary file, the *claim procedure code* attributes.

These are attributes that I initially guessed would be useful in identifying potential fraudulence. My rationale was that certain procedures might be easier to fake than others, so these codes that identify them might then be useful predictors.

In preliminary checks of the data, I soon learned the contrary: these _Claim Procedure Code_ attributes are virtually useless, as they are all mostly unpopulated. They are sparse enough that I decided to drop them without much hesitation.

I expose this sparsity in the code cells that follow, reenacting my cursory checks:

**Identifying and listing these attributes**

These _Claim Procedure Code_ attributes can be indicated by whether their name contains the substring, 'Procedure'.

Creating a list of these attributes:

In [ ]:
claimProc_attrs = [x for x in df.columns.tolist() if 'Procedure' in x]
claimProc_attrs

['ClmProcedureCode_1',
 'ClmProcedureCode_2',
 'ClmProcedureCode_3',
 'ClmProcedureCode_4',
 'ClmProcedureCode_5',
 'ClmProcedureCode_6']

Checking these attributes' counts of null values:

In [ ]:
claimproc_nulls_df = df[claimProc_attrs].isnull().sum(axis = 0)
claimproc_nulls_df

,0
ClmProcedureCode_1,534901
ClmProcedureCode_2,552721
ClmProcedureCode_3,557242
ClmProcedureCode_4,558093
ClmProcedureCode_5,558202
ClmProcedureCode_6,558211


Checking their counts of populated values:

In [ ]:
claimproc_notnulls_df = df[claimProc_attrs].notnull().sum(axis = 0)
df[claimProc_attrs].notnull().sum(axis = 0)

,0
ClmProcedureCode_1,23310
ClmProcedureCode_2,5490
ClmProcedureCode_3,969
ClmProcedureCode_4,118
ClmProcedureCode_5,9
ClmProcedureCode_6,0


Calculating the proportion of populated values in the most populated of these attributes, *ClmProcedureCode_1*:

In [ ]:
claimproc_proportion_popld = claimproc_notnulls_df / (claimproc_nulls_df + claimproc_notnulls_df)

pd.concat([claimproc_nulls_df, claimproc_notnulls_df, claimproc_proportion_popld],
          axis=1).set_axis(['Null counts', 'Populated counts', 'Portion populated'], axis=1)

,Null counts,Populated counts,Portion populated
ClmProcedureCode_1,534901,23310,0.041758
ClmProcedureCode_2,552721,5490,0.009835
ClmProcedureCode_3,557242,969,0.001736
ClmProcedureCode_4,558093,118,0.000211
ClmProcedureCode_5,558202,9,0.000016
ClmProcedureCode_6,558211,0,0.000000


Even the highest proportion of populated values is terribly sparse- that of *ClmProcedureCode_1* showing 4.18%.

With this being so, I do not attempt any aggregation of these features, and thus leave them to be discarded in the next section, where the integration process concludes.



---


### *Saving objects to file for part #4*

In [ ]:
filesave_path = project_dir_path + '/walkthrough/part_3'
!mkdir -p {filesave_path}

df.to_pickle(f'{filesave_path}/df.pkl')